# Import and High-Level Setup

In [1]:
# General Python Packages
import os
import time

# Torch Packages
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch.optim import lr_scheduler, SGD
from torch.autograd import Variable
from torch import nn
from torch.nn import DataParallel
from torch.nn import Module

# General Analytics Packages
import pandas as pd
import numpy as np

# Visualization / Image Packages
import matplotlib.pyplot as plt
from PIL import Image

# Randomization Functions
from random import random as randuni

In [2]:
# Put MatPlotLib in interactive mode
plt.ion()

# Define Data Manipulation Classes

### Helper Utility Classes

In [3]:
def is_image_file(fname):
    """Checks if a file is an image.
    Args:
        fname (string): path to a file
    Returns:
        bool: True if the filename ends with a known image extension
    """
    return fname.lower().endswith('.png')

def create_label_maps(details_df):
    """ Take a descriptive dataframe and extract the unique labels and map to index values
    Args:
        details_df: Dataframe with the image details
    Returns:
        label_list: list of unique labels in the dataframe
        label_to_index: map from labels to indices
    """
    """ TODO: Research paper also excludes these labels but need to figure out how to handle
              cases that have these as positive findings (completely exclude?)
    excluded_labels = ['Edema','Hernia','Emphysema','Fibrosis','No Finding'
                      'Pleural_Thickening','Consolidation']
    """
    excluded_labels = ['No Finding']
    
    label_groups = details_df['Finding Labels'].unique()
    unique_labels = set([label for sublist in label_groups.tolist() for label in sublist.split('|')])
    
    # Drop some label that we do not want to include
    unique_labels = [l for l in unique_labels if l not in excluded_labels]

    index_to_label = {idx: val for idx, val in enumerate(unique_labels)}
    label_to_index = {val: idx for idx, val in index_to_label.items()}

    label_list = list(label_to_index.keys())

    return label_list, label_to_index

def create_image_list(dir):
    """ Create a full list of images available 
    Args:
        dir (string): root directory of images with subdirectories underneath
                      that have the .png images within them
    Returns:
        image_list: list of tuples with (image_name, full_image_path)
    """
    image_list = []
    dir = os.path.expanduser(dir)
    for subfolder in sorted(os.listdir(dir)):
        d = os.path.join(dir, subfolder)
        if not os.path.isdir(d):
            continue
        for subfolder_path, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                if is_image_file(fname):
                    path = os.path.join(subfolder_path, fname)
                    image_list.append((fname, path))
    return image_list

def pil_loader(path):
    """ Opens path as file with Pillow (https://github.com/python-pillow/Pillow/issues/835)
    Args:
        path (string): File path to the image
    Returns:
        img: Image in RGB format
    """
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
        
def imshow(inp, title=None):
    """ Convert tensor array to an image (only use post-dataset transform) """
    inp = inp[0]
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

### Implementation of Torch's Dataset

In [4]:
class XrayImageSet(Dataset):
    """
    Args:
        image_root (string): root directory of the images in form image/subfolder/*.png
        csv_file (string): path to the CSV data file
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
     Attributes:
        labels (list): list of the possible label names.
        label_to_index (dict): look from label name to a label index
        imgs (list): List of (filename, image path) tuples
    """
    
    def __init__(self, image_root, csv_file, transform=None, target_transform=None, loader = pil_loader):
        """ Create an instance of the Xray Dataset """
        img_details = pd.read_csv(csv_file)
        
        labels, label_to_index = create_label_maps(img_details)
        imgs = create_image_list(image_root)

        self.imgs = imgs
        self.image_details = img_details
        self.image_root = image_root
        self.labels = labels
        self.label_to_index = label_to_index
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        self.max_label_index = max(label_to_index.values())

    def __getitem__(self, index):
        """ Get image,labels pair by index
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is class_index of the target class.
        """
        fname, path = self.imgs[index]
        target = self.get_one_hot_labels(fname)
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        """ Calculate length of the dataset (number of images) """
        return len(self.imgs)
    
    def get_labels(self, fname):
        """ Return the label string for the file """
        return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]
    
    def one_hot_labels(self, labels):
        """ Convert the labels string (with each label separated by |) into 1-hot encoding """
        if labels == None:
            return None
        
        split_label_indices = [self.label_to_index.get(label)
                               for label in labels.split('|')
                               if label != 'No Finding']
        
        out = [1 if idx in split_label_indices else 0 for idx in range(self.max_label_index+1)]
        # This code UNHOTs the labels:
        # out = '|'.join([index_to_label.get(idx) for idx, val in enumerate(one_hot_tuple) if val == 1])
        return out

    def get_one_hot_labels(self, fname):
        """ Get the 1-hot encoded label array for the provided file """
        labels = self.get_labels(fname)
        one_hot_labels = self.one_hot_labels(labels)
        return torch.FloatTensor(one_hot_labels)

### Create the dataset with necessary transformations

In [5]:
img_transforms = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

In [6]:
img_data_train = XrayImageSet(image_root = '/user/images/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms,
                              target_transform = None)

img_data_train.imgs = [img for i, img in enumerate(img_data_train.imgs) if i % 10 > 0]# and randuni() < 0.1]

In [7]:
img_data_val   = XrayImageSet(image_root = '/user/images/',
                              csv_file = '/user/img_details.csv',
                              transform = img_transforms,
                              target_transform = None)

img_data_val.imgs = [img for i, img in enumerate(img_data_val.imgs) if i % 10 == 0]# and randuni() < 0.1]

In [8]:
print("Training Set Size: {}".format(len(img_data_train)))
print("Validation Set Size: {}".format(len(img_data_val)))

Training Set Size: 100908
Validation Set Size: 11212


### Put the dataset into a Dataloader to handle batching

In [153]:
batch_size = 5
num_gpus = torch.cuda.device_count()
pin_mem_setting = True

print("Number of GPU: {}".format(num_gpus))

Number of GPU: 1


In [154]:
img_loader_train = DataLoader(img_data_train,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = 10,
                              pin_memory = pin_mem_setting)

img_loader_val   = DataLoader(img_data_val,
                              batch_size = batch_size * num_gpus,
                              shuffle = True,
                              num_workers = 10,
                              pin_memory = pin_mem_setting)

In [155]:
dataloaders = {
    'train': img_loader_train,
    'val': img_loader_val
}

# Define model training procedure

In [12]:
class printer_writer:
    def __init__(self, output_folder_path):
        self.start_time = time.strftime('%Y%m%d-%Hh%Mm%Ss')
        
        self.outprefix = output_folder_path + '/' + self.start_time
        
        # Print Output File
        self.print_out_path = self.outprefix + '_print.txt'
        self.print_out_file = open(self.print_out_path, 'w', 1)
        
    def printw(self, string):
        print(string)
        try:
            self.print_out_file.write(string + "\n")
        except: # Ignore errors
            pass
        
    def save_checkpoint(self, epoch, model, optimizer, scheduler, val_error):
        model_out_path = self.outprefix + '_model_' + str(epoch+1) + '.tar'
        
        torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
        
    def close(self):
        self.print_out_file.close()

In [13]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, outfolder = '/user/xrayproj/output/'):
    since = time.time()
    scribe = printer_writer(outfolder)

    for epoch in range(num_epochs):
        scribe.printw('Epoch {}/{}'.format(epoch, num_epochs - 1))
        scribe.printw('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0
            obs_counter = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # Store statistics (convert from autograd.Variable to float/int)
                loss_val = loss.data[0]
                correct_val = torch.sum( ((outputs.sigmoid()>0.5) == (labels>0.5)).long() ).data[0]
                
                running_loss += loss_val
                running_corrects += correct_val
                
                obs_counter += len(inputs)
                
                batch_loss = 1.0 * loss_val / len(inputs)
                batch_acc = 1.0 * correct_val / len(inputs)
                status = ' |~~ {}@{}  Loss: {:.6f} Acc: {:.4f}'.format(
                    phase, obs_counter, batch_loss, batch_acc)
                scribe.printw(status)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            scribe.printw('{}  Loss: {:.6f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val':
                scribe.save_checkpoint(epoch, model, optimizer, scheduler, epoch_loss)

    time_elapsed = time.time() - since
    scribe.printw('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    scribe.close()

    return model

# Define Weighted Cost Metrics

In [14]:
def imbalance_weighted_bce_with_logit(input, target, size_average=True):
    if not (target.size() == input.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

    max_val = (-input).clamp(min=0)
    loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

    # Determine |P| and |N|
    positive_labels = target.sum()
    negative_labels = (1-target).sum()

    # Upweight the less common class (very often the 1s)
    beta_p = (positive_labels + negative_labels) / positive_labels
    beta_n = (positive_labels + negative_labels) / negative_labels

    # Adjust the losses accordingly
    loss_weight = target * beta_p + (1-target) * beta_n
    
    loss = loss * loss_weight

    if size_average:
        return loss.mean()
    else:
        return loss.sum()

In [15]:
class BCEWithLogitsImbalanceWeightedLoss(Module):
    def __init__(self, class_weight=None, size_average=True):
        super(BCEWithLogitsImbalanceWeightedLoss, self).__init__()
        self.size_average = size_average

    def forward(self, input, target):
        return imbalance_weighted_bce_with_logit(input, target, size_average=self.size_average)

# Setup Neural Network

### Define the model specifications

In [16]:
def ResNet18PlusFlexibleFC():
    # Create a base ResNet18 model
    m = models.resnet18(pretrained=True)
    for param in m.parameters():
        param.requires_grad = False

    # Replace the final FC layer
    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

In [17]:
def ResNet18PlusFCFullyFlexible():
    # Create a base ResNet18 model
    m = models.resnet18(pretrained=True)

    # Replace the final FC layer
    m.fc = nn.Linear(m.fc.in_features, len(img_data_train.labels))
    
    return m

### Pull the ResNet-18 pre-trained model and replace the fully connected layer at the end

In [156]:
#model_base = ResNet18PlusFlexibleFC()
model_base = ResNet18PlusFCFullyFlexible()

### Push model to CUDA/GPU

In [157]:
model_ft = DataParallel(model_base).cuda()

### Define loss measure and learning rates/procedures

In [158]:
#criterion = BCEWithLogitsImbalanceWeightedLoss()
criterion_base = nn.BCEWithLogitsLoss()

# Observe that all parameters are being optimized
optimizer_ft = SGD(model_ft.module.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [159]:
criterion = criterion_base.cuda()

### Future code for allowing optimization of the base layer with a lower learning rate

```
ignored_params = list(map(id, model.fc.parameters()))
base_params = filter(lambda p: id(p) not in ignored_params,
                     model.parameters())

optimizer = torch.optim.SGD([
            {'params': base_params},
            {'params': model.fc.parameters(), 'lr': opt.lr}
        ], lr=opt.lr*0.1, momentum=0.9)
```

# Begin Training Network (Normal Cost)

In [135]:
d = Variable(img_data_train[0][0].unsqueeze(0))
act = Variable(torch.FloatTensor([0,0,0,0,1,0,0,0,0,0,0,0,0,0]).unsqueeze(0).cuda())
out = model_ft.forward(d)

In [132]:
loss = criterion(out, act)

In [133]:
loss.backward()

In [160]:
train_model(model_ft,
            criterion,
            optimizer_ft,
            exp_lr_scheduler,
            num_epochs=10)

Epoch 0/9
----------
 |~~ train@5  Loss: 0.130731 Acc: 8.8000
 |~~ train@10  Loss: 0.129181 Acc: 8.6000
 |~~ train@15  Loss: 0.131137 Acc: 8.8000
 |~~ train@20  Loss: 0.119419 Acc: 9.8000
 |~~ train@25  Loss: 0.124706 Acc: 9.4000
 |~~ train@30  Loss: 0.117774 Acc: 10.4000
 |~~ train@35  Loss: 0.117114 Acc: 10.8000
 |~~ train@40  Loss: 0.109142 Acc: 11.6000
 |~~ train@45  Loss: 0.105148 Acc: 11.0000
 |~~ train@50  Loss: 0.102704 Acc: 11.2000
 |~~ train@55  Loss: 0.096036 Acc: 11.8000
 |~~ train@60  Loss: 0.093051 Acc: 12.0000
 |~~ train@65  Loss: 0.082448 Acc: 13.0000
 |~~ train@70  Loss: 0.086912 Acc: 12.4000
 |~~ train@75  Loss: 0.073407 Acc: 13.4000
 |~~ train@80  Loss: 0.072460 Acc: 13.2000
 |~~ train@85  Loss: 0.079333 Acc: 12.6000
 |~~ train@90  Loss: 0.071389 Acc: 12.8000
 |~~ train@95  Loss: 0.059408 Acc: 13.8000
 |~~ train@100  Loss: 0.072374 Acc: 12.6000
 |~~ train@105  Loss: 0.048388 Acc: 14.0000
 |~~ train@110  Loss: 0.060040 Acc: 13.2000
 |~~ train@115  Loss: 0.049411 Acc: 

 |~~ train@945  Loss: 0.037917 Acc: 13.4000
 |~~ train@950  Loss: 0.021500 Acc: 13.8000
 |~~ train@955  Loss: 0.024782 Acc: 13.4000
 |~~ train@960  Loss: 0.013406 Acc: 14.0000
 |~~ train@965  Loss: 0.040621 Acc: 13.0000
 |~~ train@970  Loss: 0.013110 Acc: 14.0000
 |~~ train@975  Loss: 0.052959 Acc: 12.8000
 |~~ train@980  Loss: 0.035436 Acc: 13.2000
 |~~ train@985  Loss: 0.044536 Acc: 13.0000
 |~~ train@990  Loss: 0.052441 Acc: 13.0000
 |~~ train@995  Loss: 0.036882 Acc: 13.0000
 |~~ train@1000  Loss: 0.020807 Acc: 13.8000
 |~~ train@1005  Loss: 0.020710 Acc: 13.8000
 |~~ train@1010  Loss: 0.035231 Acc: 13.4000
 |~~ train@1015  Loss: 0.044559 Acc: 13.0000
 |~~ train@1020  Loss: 0.059630 Acc: 12.8000
 |~~ train@1025  Loss: 0.049210 Acc: 13.0000
 |~~ train@1030  Loss: 0.038071 Acc: 13.2000
 |~~ train@1035  Loss: 0.026906 Acc: 13.6000
 |~~ train@1040  Loss: 0.055385 Acc: 12.8000
 |~~ train@1045  Loss: 0.020046 Acc: 13.8000
 |~~ train@1050  Loss: 0.037808 Acc: 13.0000
 |~~ train@1055  Loss

 |~~ train@1860  Loss: 0.032888 Acc: 13.2000
 |~~ train@1865  Loss: 0.022463 Acc: 13.6000
 |~~ train@1870  Loss: 0.014320 Acc: 13.8000
 |~~ train@1875  Loss: 0.019739 Acc: 13.6000
 |~~ train@1880  Loss: 0.038914 Acc: 13.2000
 |~~ train@1885  Loss: 0.024890 Acc: 13.4000
 |~~ train@1890  Loss: 0.050289 Acc: 13.0000
 |~~ train@1895  Loss: 0.037360 Acc: 13.2000
 |~~ train@1900  Loss: 0.041308 Acc: 13.2000
 |~~ train@1905  Loss: 0.010676 Acc: 14.0000
 |~~ train@1910  Loss: 0.058453 Acc: 12.8000
 |~~ train@1915  Loss: 0.050916 Acc: 13.2000
 |~~ train@1920  Loss: 0.054504 Acc: 13.0000
 |~~ train@1925  Loss: 0.015961 Acc: 13.8000
 |~~ train@1930  Loss: 0.017232 Acc: 13.8000
 |~~ train@1935  Loss: 0.018023 Acc: 13.8000
 |~~ train@1940  Loss: 0.018406 Acc: 13.8000
 |~~ train@1945  Loss: 0.027744 Acc: 13.4000
 |~~ train@1950  Loss: 0.034644 Acc: 13.4000
 |~~ train@1955  Loss: 0.017766 Acc: 13.8000
 |~~ train@1960  Loss: 0.041941 Acc: 13.2000
 |~~ train@1965  Loss: 0.024225 Acc: 13.6000
 |~~ train

 |~~ train@2795  Loss: 0.021227 Acc: 13.6000
 |~~ train@2800  Loss: 0.050143 Acc: 13.2000
 |~~ train@2805  Loss: 0.018437 Acc: 13.8000
 |~~ train@2810  Loss: 0.041060 Acc: 13.2000
 |~~ train@2815  Loss: 0.062830 Acc: 12.8000
 |~~ train@2820  Loss: 0.029862 Acc: 13.4000
 |~~ train@2825  Loss: 0.025088 Acc: 13.4000
 |~~ train@2830  Loss: 0.022825 Acc: 13.6000
 |~~ train@2835  Loss: 0.024881 Acc: 13.6000
 |~~ train@2840  Loss: 0.052559 Acc: 13.0000
 |~~ train@2845  Loss: 0.055728 Acc: 13.0000
 |~~ train@2850  Loss: 0.019223 Acc: 13.6000
 |~~ train@2855  Loss: 0.037188 Acc: 13.2000
 |~~ train@2860  Loss: 0.027191 Acc: 13.6000
 |~~ train@2865  Loss: 0.042322 Acc: 13.0000
 |~~ train@2870  Loss: 0.059598 Acc: 12.6000
 |~~ train@2875  Loss: 0.016192 Acc: 13.8000
 |~~ train@2880  Loss: 0.051062 Acc: 13.0000
 |~~ train@2885  Loss: 0.049392 Acc: 13.0000
 |~~ train@2890  Loss: 0.047780 Acc: 12.8000
 |~~ train@2895  Loss: 0.025760 Acc: 13.6000
 |~~ train@2900  Loss: 0.039874 Acc: 13.4000
 |~~ train

 |~~ train@3745  Loss: 0.019962 Acc: 13.6000
 |~~ train@3750  Loss: 0.042285 Acc: 13.0000
 |~~ train@3755  Loss: 0.032067 Acc: 13.2000
 |~~ train@3760  Loss: 0.024246 Acc: 13.4000
 |~~ train@3765  Loss: 0.010064 Acc: 14.0000
 |~~ train@3770  Loss: 0.025117 Acc: 13.6000
 |~~ train@3775  Loss: 0.018830 Acc: 13.6000
 |~~ train@3780  Loss: 0.028907 Acc: 13.6000
 |~~ train@3785  Loss: 0.010196 Acc: 14.0000
 |~~ train@3790  Loss: 0.069163 Acc: 12.6000
 |~~ train@3795  Loss: 0.030617 Acc: 13.6000
 |~~ train@3800  Loss: 0.053657 Acc: 13.0000
 |~~ train@3805  Loss: 0.025226 Acc: 13.6000
 |~~ train@3810  Loss: 0.040118 Acc: 13.2000
 |~~ train@3815  Loss: 0.009613 Acc: 14.0000
 |~~ train@3820  Loss: 0.027436 Acc: 13.6000
 |~~ train@3825  Loss: 0.013880 Acc: 13.8000
 |~~ train@3830  Loss: 0.027754 Acc: 13.6000
 |~~ train@3835  Loss: 0.026370 Acc: 13.4000
 |~~ train@3840  Loss: 0.029846 Acc: 13.6000
 |~~ train@3845  Loss: 0.023835 Acc: 13.6000
 |~~ train@3850  Loss: 0.022630 Acc: 13.6000
 |~~ train

...

Process Process-29:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Process-25:
Process Process-23:
Process Process-21:
Traceback (most recent call last):
Process Process-26:
Traceback (most recent call last):
Process Process-28:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "<ipython-input-3-4cc34eea5387>", line 68, in pil_loader
    return img.convert('RGB')
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/an

 |~~ train@37355  Loss: 0.020940 Acc: 13.8000
 |~~ train@37360  Loss: 0.046885 Acc: 13.0000
 |~~ train@37365  Loss: 0.024683 Acc: 13.6000
 |~~ train@37370  Loss: 0.015186 Acc: 13.8000
 |~~ train@37375  Loss: 0.037839 Acc: 13.2000
 |~~ train@37380  Loss: 0.045156 Acc: 13.0000
 |~~ train@37385  Loss: 0.030542 Acc: 13.4000
 |~~ train@37390  Loss: 0.022626 Acc: 13.6000


  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 860, in convert
    self.load()
Process Process-30:
  File "<ipython-input-3-4cc34eea5387>", line 68, in pil_loader
    return img.convert('RGB')
  File "<ipython-input-3-4cc34eea5387>", line 68, in pil_loader
    return img.convert('RGB')
  File "<ipython-input-4-21cc0ac35a56>", line 42, in __getitem__
    target = self.get_one_hot_labels(fname)
  File "<ipython-input-4-21cc0ac35a56>", line 45, in __getitem__
    img = self.transform(img)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/ImageFile.py", line 234, in load
    n, err_code = decoder.decode(b)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/PIL/Image.py", line 860, in convert
    self.load()
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-4-21cc0ac35a56>", line 75, in ge

KeyboardInterrupt: 

Process Process-22:
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py", line 2175, in _getitem_array
    return self._take(indexer, axis=0, convert=False)
  File "<ipython-input-4-21cc0ac35a56>", line 75, in get_one_hot_labels
    labels = self.get_labels(fname)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py", line 2150, in _take
    verify=True)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 4259, in take
    axis=axis, allow_dups=True)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 4145, in reindex_indexer
    for blk in self.blocks]
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/internals.py", line 4145, in <listcomp>
    for blk in self.blocks]
  File "<ipython-input-4-21cc0ac35a56>", line 57, in get_labels
    return self.image_details[self.image_details['Image Index'] == fname]['Finding Labels'].values[0]


### Save model results to S3

aws s3 cp ResNet18PlusFlexibleFC_Epoch9.tar s3://bdh-xrayproj-modelparameters/

import boto3

s3 = boto3.client('s3')
s3.list_buckets()

S3 Commands: http://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html

Boto3 QuickStart: http://boto3.readthedocs.io/en/latest/guide/quickstart.html

Key Management: https://aws.amazon.com/blogs/security/a-safer-way-to-distribute-aws-credentials-to-ec2/

AWS IAM Rules: http://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_use_switch-role-api.html

### Load model results back from S3

In [26]:
exp_lr_scheduler.last_epoch

9

# Analysis of Model Results

In [54]:
out_model_30.train(mode=False)

obs_counter = 0
total_pred = Variable(torch.FloatTensor(torch.zeros(14)))
total_act = Variable(torch.FloatTensor(torch.zeros(14)))

conf_a = {}
conf_b = {}
conf_c = {}
conf_d = {}
for i in range(1,10):
    conf_a[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_b[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_c[i] = Variable(torch.FloatTensor(torch.zeros(14)))
    conf_d[i] = Variable(torch.FloatTensor(torch.zeros(14)))

for data in dataloaders['val']:
    print("STARTING ITERATION...")
    inputs, labels = data
    print("PROCESSING FIRST {} OBSERVATIONS".format(len(inputs)))

    inputs = Variable(inputs.cuda())
    labels = Variable(labels.cuda())

    outputs = out_model_30(inputs).sigmoid()
    
    total_act += labels.sum(0).cpu()
    total_pred += outputs.sum(0).cpu()

    # Store statistics (convert from autograd.Variable to float/int)
    for i in range(1,10):
        t = i/10
        conf_a[i] += ((outputs.sigmoid()>t) == (labels>0.5)).sum(0).cpu().float()
        conf_b[i] += ((outputs.sigmoid()<t) == (labels>0.5)).sum(0).cpu().float()
        conf_c[i] += ((outputs.sigmoid()>t) == (labels<0.5)).sum(0).cpu().float()
        conf_d[i] += ((outputs.sigmoid()<t) == (labels<0.5)).sum(0).cpu().float()

    obs_counter += len(inputs)

STARTING ITERATION...
PROCESSING FIRST 750 OBSERVATIONS
STARTING ITERATION...
PROCESSING FIRST 406 OBSERVATIONS


In [67]:
comparison = Variable(torch.FloatTensor(9, 14))
for i in range(9):
    comparison[0] = conf_a[1] / obs_counter
print(comparison.int())

Variable containing:

Columns 0 to 5 
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -2.1475e+09  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00 -2.1475e+09  0.0000e+00 -2.1475e+09  0.0000e+00
 6.2634e+06 -1.0000e+00 -2.1475e+09  0.0000e+00 -2.1475e+09  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00

Columns 6 to 11 
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -6.5460e+04  0.0000e+00
 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00
-2.1475e

In [83]:
conf_d[9] / obs_counter

Variable containing:
 0.3183
 0.1462
 0.3356
 0.2171
 0.3166
 0.3183
 0.2803
 0.2898
 0.2232
 0.3279
 0.2846
 0.3045
 0.3002
 0.3192
[torch.FloatTensor of size 14]

In [84]:
img_data_train.labels

['Fibrosis',
 'Infiltration',
 'Hernia',
 'Effusion',
 'Emphysema',
 'Edema',
 'Cardiomegaly',
 'Mass',
 'Nodule',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Thickening',
 'Consolidation',
 'Pneumonia']

In [101]:
'''
torch.save({
            'epoch': epoch+1,
            'state': model.state_dict(),
            'optimizer': optimizer,
            'scheduler': scheduler,
            'val_error': val_error
        }, model_out_path)
'''
test_load = torch.load('/user/xrayproj/output/20171120-01h41m56s_model_9.tar')

In [103]:
test_load.keys()

dict_keys(['epoch', 'state', 'optimizer', 'scheduler', 'val_error'])

In [106]:
load_opt = test_load['optimizer']
load_sched = test_load['scheduler']
load_state = test_load['state']

In [117]:
model2 = models.resnet18(pretrained=True)
for param in model2.parameters():
    param.requires_grad = False

# Replace FC layer
model2.fc = nn.Linear(model2.fc.in_features, len(img_data_train.labels))

model2_c = DataParallel(model2).cuda()

In [118]:
model2_c.load_state_dict(load_state)

In [123]:
model2_c.forward(Variable(img_data_train[0][0].unsqueeze(0).cuda())).sigmoid()

Variable containing:

Columns 0 to 9 
 0.7873  0.5093  0.2980  0.6386  0.4665  0.3371  0.1614  0.1846  0.1925  0.6630

Columns 10 to 13 
 0.0288  0.4468  0.3684  0.2549
[torch.cuda.FloatTensor of size 1x14 (GPU 0)]